In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [3]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5, min_tracking_confidence = 0.5)
mp_drawing = mp.solutions.drawing_utils
drwaing_spec = mp_drawing.DrawingSpec(thickness = 1, circle_radius = 1)


In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    
    if not success:
        continue
    landmarkImg = image.copy()
    start = time.time()
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image.flags.writeable = False
    
    results = face_mesh.process(image)
    
    image.flags.writeable = True
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    img_h, img_w, img_c = image.shape
    
    face_3d= []
    
    face_2d = []
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(landmarkImg,face_landmarks) 
            for idx, lm in enumerate(face_landmarks.landmark):
                
               
                
                if idx==33 or idx==263 or idx==1 or idx==61 or idx==291 or idx==199:
                    if idx==1:
                        nose_2d = (lm.x*img_w, lm.y*img_h)
                        nose_3d = (lm.x*img_w, lm.y*img_h, lm.z * 3000)
                                   
                    x,y = int(lm.x*img_w), int(lm.y*img_h)
                    #print(x,y)        
                    face_2d.append([x,y])
                    face_3d.append([x,y,lm.z])
                                   
            
            face_2d = np.array(face_2d, dtype = np.float32)
            face_3d = np.array(face_3d, dtype = np.float32)
            print(type(face_2d[0][0]))
            focal_length = 1*img_w
                                   
            cam_matrix = np.array([[focal_length,0,img_h/2],
                                  [0,focal_length,img_w/2],
                                   [0,0,1]
                                  ], dtype= np.float32)
            
            dist_matrix = np.zeros((4,1), dtype = np.float32)
                                   
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d,face_2d,cam_matrix,dist_matrix)                     
                                   
            rmat, jac = cv2.Rodrigues(rot_vec)
            
            angles, mtxr, mtxq, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
                                   
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            text = "Person is "
            text2 = ""
            if y<-10:
                text += "Looking left"
            elif x<-10:
                text += "Looking Down\n"
                text2 += "Driver might be drowsy"
            elif y>10:
                text += "Looking Right"            
            elif x>10:
                text += "Looking Up"
            else:
                text += "looking Forward"
                                   
            nose_3d_projection , jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
                                   
            p1 = int(nose_2d[0]), int(nose_2d[1])
            p2 = int(nose_2d[0] + y*10), int(nose_2d[1] - x*10)
                                   
            cv2.line(image, p1, p2, (0,255,0),3)
                                   
            cv2.putText(image,text,(20,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            if len(text2)!=0:
                cv2.putText(image,text2,(20,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        
        end = time.time()
        
        fps = 1/(end - start+0.0001)
        
        print("fps : ", fps)  
        fps = str(fps)
        cv2.putText(image,"FPS : "+fps,(20,450),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
    cv2.imshow("Landmarks ",landmarkImg)
    cv2.imshow("image", image)
    if cv2.waitKey(1) & 0xFF ==27:
        break
                                   
cap.release()
cv2.destroyAllWindows()

<class 'numpy.float32'>
fps :  11.75064560214862
<class 'numpy.float32'>
fps :  43.30076564410376
<class 'numpy.float32'>
fps :  41.48281701723443
<class 'numpy.float32'>
fps :  49.76663914425316
<class 'numpy.float32'>
fps :  49.821024113357424
<class 'numpy.float32'>
fps :  49.770772957320574
<class 'numpy.float32'>
fps :  21.693430689069384
<class 'numpy.float32'>
fps :  52.6093616422797
<class 'numpy.float32'>
fps :  28.497895314588742
<class 'numpy.float32'>
fps :  55.26133698099532
<class 'numpy.float32'>
fps :  49.77904264402248
<class 'numpy.float32'>
fps :  52.36633324839465
<class 'numpy.float32'>
fps :  56.03056340015647
<class 'numpy.float32'>
fps :  47.412321325475915
<class 'numpy.float32'>
fps :  27.679249840633453
<class 'numpy.float32'>
fps :  52.4586815141641
<class 'numpy.float32'>
fps :  55.315267949999004
<class 'numpy.float32'>
fps :  34.41411577380942
<class 'numpy.float32'>
fps :  55.41246301426728
<class 'numpy.float32'>
fps :  43.29897762026709
<class 'numpy.f